# Genetic Algorithm

This notebook showcases how to use the built-in Genetic Algorithm (GA)

In [1]:
using Statistics
using EvoLP
using OrderedCollections
using DataFrames

For this example we will use the **Rosenbrock** function:

In [2]:
@doc rosenbrock

```
rosenbrock(x; b=100)
```

!!! compat "Changed since EvoLP 1.3"
    This is the $d$-dimensional Rosenbrock function. In previous releases, it was 2d only and had an additional keyword argument `a`.

    Update your workflow accordingly.


The $d$-dimensional **Rosenbrock** *banana* benchmark function. With $b=100$, minimum is at $f([1, \dots, 1]) = 0$

$$
f(x) = \sum_{i=1}^{d-1} \left[b(x_{i+1} - x_i^2)^2 + (x_i - 1)^2 \right]
$$


In a GA, we use vectors as _individuals_.

Let's start creating the population. For that, we can use a generator. Let's use the uniform generator:

In [3]:
@doc unif_rand_vector_pop

```
unif_rand_vector_pop(n, lb, ub; rng=Random.GLOBAL_RNG)
```

Generate a population of `n` vector individuals using a uniformly random distribution between lower bounds `lb` and upper bounds `ub`.

Both `lb` and `ub` must be arrays of the same dimensions.

# Examples

```julia
julia> unif_rand_vector_pop(3, [-1, -1], [1, 1])
3-element Vector{Vector{Float64}}:
 [-0.16338687344459046, 0.31576097298524064]
 [-0.941510876597899, 0.8219576462978224]
 [-0.377090051761797, -0.28434454028992096]
```


In [4]:
pop_size = 30
population = unif_rand_vector_pop(pop_size, [-2.048, -2.048], [2.048, 2.048])
first(population, 3)

3-element Vector{Vector{Float64}}:
 [0.013224350621951597, -1.3678998860345195]
 [0.7605835675636268, -0.2944462429647383]
 [1.42885988866536, 1.3307125626289422]

In a GA, we have _selection_, _crossover_ and _mutation_.

We can easily set up these operators using the built-ins provided by EvoLP.

Let's use rank based selection and interpolation crossover:

In [5]:
@doc InterpolationCrossover

Interpolation crossover with scaling parameter `λ`.


In [6]:
S = RankBasedSelectionGenerational()
C = InterpolationCrossover(0.5)

InterpolationCrossover(0.5)

For mutation, we can use a Gaussian approach:

In [7]:
@doc GaussianMutation

Gaussian mutation with standard deviation `σ`, which should be a real number.


In [8]:
M = GaussianMutation(0.01)

GaussianMutation(0.01)

We can use the default `Logbook` to record many statistics about our run:

In [9]:
thelogger = Logbook()

Logbook(LittleDict{AbstractString, Function, Vector{AbstractString}, Vector{Function}}("min_f" => minimum, "mean_f" => Statistics.mean, "median_f" => Statistics.median, "max_f" => maximum, "std" => Statistics.std), NamedTuple{(:min_f, :mean_f, :median_f, :max_f, :std)}[])

And now we're ready to use the `GA` built-in algorithm:

In [10]:
@doc GA

```
GA(f, pop, k_max, S, C, M)
GA(logbook::Logbook, f, population, k_max, S, C, M)
GA(notebooks::Vector{Logbook}, f, population, k_max, S, C, M)
```

Generational Genetic Algorithm.

## Arguments

  * `f::Function`: objective function to **minimise**.
  * `population::AbstractVector`: a list of vector individuals.
  * `k_max::Integer`: number of iterations.
  * `S::SelectionMethod`: one of the available [`SelectionMethod`](@ref).
  * `C::CrossoverMethod`: one of the available [`CrossoverMethod`](@ref).
  * `M::MutationMethod`: one of the available [`MutationMethod`](@ref).

Returns a [`Result`](@ref).


In [11]:
result = GA(thelogger, rosenbrock, population, 100, S, C, M);

The output was suppressed so that we can analyse each part of the result separately:

In [12]:
@show optimum(result)

@show optimizer(result)

@show f_calls(result)

@show runtime(result)

@show thelogger.records[end]

optimum(result) = 0.15049342789954512
optimizer(result) = 

[0.5858877850983488, 0.3651832142159951]
f_calls(result) = 3000
runtime(result) = 0.757843669
thelogger.records[end] = (min_f = 0.15049342789954512, mean_f = 0.18900618533625962, median_f = 0.17928194152771926, max_f = 0.31767976898711214, std = 0.03682181655004641)


(min_f = 0.15049342789954512, mean_f = 0.18900618533625962, median_f = 0.17928194152771926, max_f = 0.31767976898711214, std = 0.03682181655004641)

For further analysis, we can export the records directly to a DataFrame:

In [13]:
DataFrame(thelogger.records)

Row,min_f,mean_f,median_f,max_f,std
,Float64,Float64,Float64,Float64,Float64
1,0.424001,331.463,139.988,1627.38,439.756
2,0.321719,84.7211,35.0588,808.533,152.371
3,0.578831,29.2711,10.1089,126.64,37.2353
4,0.275801,16.4307,7.51019,56.5675,17.7872
5,0.568604,10.9749,5.36025,46.42,11.7371
6,0.560855,5.25174,3.50996,16.0154,3.93378
7,0.673143,3.01429,2.09055,9.83939,2.31038
8,0.689797,1.65877,1.5351,2.7963,0.656081
9,0.717406,1.28313,1.21127,2.48201,0.46378
